In [5]:
## Dante Trisciuzzi
## 6.7.2023
## CS-340 SNHU

from jupyter_dash import JupyterDash
import dash_leaflet as dl
from dash import dcc, html, dash_table, Input, Output
from dash import html
import plotly.express as px
from dash import dash_table
from dash.dependencies import Input, Output
import numpy as np
import pandas as pd
from MongoCRUD import MongoCRUD

###########################
# Data Manipulation / Model
###########################

username = "aacuser"
password = "abc123"
shelter = MongoCRUD(username, password, 'AAC', 'animals')

df = pd.DataFrame.from_records(shelter.read({}))
df.drop(columns=['_id'],inplace=True)

#########################
# Dashboard Layout / View
#########################
app = JupyterDash('Dante Trisciuzzi 6-8-2023 - CS340-Mod6')

app.layout = html.Div([
    html.Div(id='hidden-div', style={'display':'none'}),
    html.Center(html.B(html.H1('Dante Trisciuzzi 6-8-2023 - CS340-Module 6'))),
    html.Hr(),
    dash_table.DataTable(
        id='datatable-id',
        columns=[
            {"name": i, "id": i, "deletable": False, "selectable": True} for i in df.columns
        ],
        data=df.to_dict('records'),
        page_size=10,  # Limit the number of rows displayed per page
        sort_action='native',  # Enable sorting
        filter_action='native',  # Enable filtering
        row_selectable='single',  # Enable single-row selection
        selected_rows=[0],  # Set an initial row selection to prevent issues
        style_table={'overflowX': 'auto'},  # Add horizontal scrollbar if table overflows
        style_cell={
            'minWidth': '0px', 'maxWidth': '180px',  # Set minimum and maximum width for cells
            'overflow': 'hidden', 'textOverflow': 'ellipsis',  # Hide cell content if it overflows
        },
    ),
    html.Br(),
    html.Hr(),
    html.Div(
        id='map-id',
        className='col s12 m6',
        children=[
            dl.Map(id='map', style={'width': '100%', 'height': '50vh', 'margin': '0 auto'}),
            ],
        ),
    ])

#############################################
# Interaction Between Components / Controller
#############################################
#This callback will highlight a row on the data table when the user selects it
@app.callback(
    Output('datatable-id', 'style_data_conditional'),
    [Input('datatable-id', 'selected_columns')]
    )
def update_styles(selected_columns):
    return [{
        'if': { 'column_id': i },
        'background_color': '#D2F3FF'
    } for i in selected_columns]


# This callback will update the geo-location chart for the selected data entry
# derived_virtual_data will be the set of data available from the datatable in the form of 
# a dictionary.
# derived_virtual_selected_rows will be the selected row(s) in the table in the form of
# a list. For this application, we are only permitting single row selection so there is only
# one value in the list.
# The iloc method allows for a row, column notation to pull data from the datatable

@app.callback(
    Output('map-id', "children"),
    [Input('datatable-id', "derived_virtual_data"),
     Input('datatable-id', "derived_virtual_selected_rows")])

def update_map(viewData, index):
    try:
        if index is None:
            row = 0
        else:
            row = index[0]

        return [
            dl.Map(
                style={'width': '100%', 'height': '500px'},
                center=[30.75, -97.48],
                zoom=10,
                children=[
                    dl.TileLayer(id="base-layer-id"),
                    dl.Marker(
                        position=[df.loc[row, "location_lat"], df.loc[row, "location_long"]],
                        children=[
                            dl.Tooltip(df.loc[row, "breed"]),
                            dl.Popup([
                                html.H1("Animal Name"),
                                html.P(df.loc[row, "name"])
                            ])
                        ]
                    )
                ]
            )
        ]
    except:
        return []


app.run_server(debug=True, host="127.0.0.1", port="2234")

Dash app running on http://127.0.0.1:2234/
